In [13]:
!pwd

/home/b/b382291/regridder/AQUA


In [14]:
import intake

In [16]:
# run mean OPA 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("meanOPA")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from meanOPA import *
from meanOPA import meanOPA # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
meanStat = meanOPA(meanFreq = "daily", save = "false")

# want to change to this 
#  stat = "mean", 


Running the algorithm

In [6]:

for i in range(0, 8328, 1): 
#
    ds1 = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    dm = meanStat.mean(ds1)


In [8]:
#a = meanStat.dsNp
b = meanStat.meanCum

c = np.zeros((1, np.size(ds1.lat), np.size(ds1.lon)))
# = xr.zeros_like(ds1)

#c = xr.DataArray.copy(a)

#check = np.subtract(a, c) 


finalMean = np.expand_dims(meanStat.meanCum, axis=0) # adding back extra time dimension 


In [26]:

if (hasattr(meanStat, 'var')): # if there are multiple variables in the file 
    # needs to convert from dataSet to a dataArray 
    fileName = meanStat.filePathSave + "_" + meanStat.var + "_" + meanStat.meanFreq + "_mean.nc" 
else: 
    fileName = meanStat.filePathSave + "_" + dm.cfVarName + "_" + meanStat.meanFreq  + "_mean.nc" 
# + self.cfVarName 

#fileName = self.filePathSave + "_" + self.meanFreq + str(finalTimeStamp) + "_mean.nc" 
dm.to_netcdf(path = fileName, mode ='w') # will re-write the file if it is already there
dm.close() 
print('finished saving')

finished saving


In [10]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

In [11]:
a= dm.meanVar - npMean
a.values

array([[[ 2.84217094e-14,  2.84217094e-14,  2.84217094e-14, ...,
          2.84217094e-14,  5.68434189e-14,  2.84217094e-14],
        [ 0.00000000e+00,  0.00000000e+00, -2.84217094e-14, ...,
          0.00000000e+00,  0.00000000e+00,  2.84217094e-14],
        [-2.84217094e-14,  2.84217094e-14, -2.84217094e-14, ...,
          2.84217094e-14, -2.84217094e-14, -2.84217094e-14],
        ...,
        [ 0.00000000e+00, -2.84217094e-14,  5.68434189e-14, ...,
          2.84217094e-14,  2.84217094e-14,  2.84217094e-14],
        [-2.84217094e-14, -2.84217094e-14,  0.00000000e+00, ...,
         -2.84217094e-14,  0.00000000e+00,  0.00000000e+00],
        [-5.68434189e-14, -5.68434189e-14, -8.52651283e-14, ...,
         -2.84217094e-14, -5.68434189e-14, -5.68434189e-14]]])

In [17]:
# from dask.distributed import LocalCluster, Client, progress
# processes = False
# nworker = 32
# ncpu = 32
# threads = ncpu // nworker
# print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
# client = Client(processes=processes,
#  threads_per_worker=threads,
#  n_workers=nworker,
#  memory_limit='64GB'
#  )

#catalogue();


os.chdir('/home/b/b382291/regridder/AQUA')

reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
data = reader.retrieve(fix=False)
tasr = reader.regrid(data["2t"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

In [95]:
loadPath = "/home/b/b382291/git/data/"
files = os.listdir(loadPath)
files = sorted(files)
savedData = xr.open_dataset(loadPath + files[9])


ds2 = tasr.isel(time=slice(216,240)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = savedData.meanVar - npMean
a.values


array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.84217094e-14,  0.00000000e+00, -2.84217094e-14],
        [ 0.00000000e+00,  2.84217094e-14, -2.84217094e-14, ...,
          2.84217094e-14,  2.84217094e-14,  5.68434189e-14],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  2.84217094e-14,  0.00000000e+00],
        ...,
        [ 2.84217094e-14,  0.00000000e+00,  2.84217094e-14, ...,
          0.00000000e+00,  2.84217094e-14,  2.84217094e-14],
        [-2.84217094e-14,  0.00000000e+00, -2.84217094e-14, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.84217094e-14,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])

In [84]:
ds2 = tasr.isel(time=slice(24,30)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = meanStat.meanCum - npMean


In [85]:
a.values

array([[-5.68434189e-14,  0.00000000e+00, -5.68434189e-14, ...,
         2.84217094e-14, -2.84217094e-14, -5.68434189e-14],
       [ 2.84217094e-14,  2.84217094e-14, -2.84217094e-14, ...,
         2.84217094e-14,  2.84217094e-14,  5.68434189e-14],
       [ 2.84217094e-14, -2.84217094e-14,  0.00000000e+00, ...,
         2.84217094e-14, -2.84217094e-14, -2.84217094e-14],
       ...,
       [ 2.84217094e-14,  0.00000000e+00, -2.84217094e-14, ...,
        -2.84217094e-14,  0.00000000e+00,  2.84217094e-14],
       [-2.84217094e-14,  0.00000000e+00,  2.84217094e-14, ...,
        -2.84217094e-14,  0.00000000e+00,  0.00000000e+00],
       [-2.84217094e-14,  0.00000000e+00,  2.84217094e-14, ...,
        -2.84217094e-14,  0.00000000e+00,  2.84217094e-14]])

In [15]:
dm

<xarray.Dataset>
Dimensions:  (time: 1, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    meanVar  (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes: (12/16)
    paramId:                    167
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfVarName:                  t2m
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid
    OPA:                        daily mean calculated using one-pass algorithm

In [19]:
tasr

<xarray.DataArray '2t' (time: 8329, lat: 900, lon: 1800)>
dask.array<reshape, shape=(8329, 900, 1800), dtype=float64, chunksize=(1, 900, 1800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2021-01-01
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
Attributes: (12/15)
    paramId:                    167
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    name:                       2 metre temperature
    cfVarName:                  t2m
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [21]:
tasr.resample(time='D').mean()

<xarray.DataArray '2t' (time: 348, lat: 900, lon: 1800)>
dask.array<stack, shape=(348, 900, 1800), dtype=float64, chunksize=(1, 900, 1800), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2021-01-01
Attributes: (12/15)
    paramId:                    167
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    name:                       2 metre temperature
    cfVarName:                  t2m
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid